In [1]:
import os
import weaviate

weaviate_api_key = os.getenv("weaviate_api_key") or "weaviate_api_key"
weaviate_url = os.getenv("weaviate_url") or "weaviate_url"
cohere_api_key = os.getenv("COHERE_API_KEY") or "COHERE_API_KEY"

auth_config = weaviate.AuthApiKey(api_key=weaviate_api_key)

client = weaviate.Client(
  url=weaviate_url,
  auth_client_secret=auth_config,
  additional_headers={"X-Cohere-Api-Key": cohere_api_key}
)
client.is_ready()

True

In [4]:
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Weaviate

index_name = "Help"
text_key = "text"
vectorstore = Weaviate(client, index_name, text_key)
vectorstore._query_attrs = ["text", "title", "category", "slug", "_additional {distance}"]
vectorstore.embedding = CohereEmbeddings(model="embed-english-v2.0", cohere_api_key=cohere_api_key)

In [8]:
query = "How much inventory should I have?"
vectorstore.similarity_search_by_text(query, k=5)

[Document(page_content='Question: How many different types, categories, or groups can the User list for both A/R and Inventory?\nAnswer: The User can list up to four different types, categories, or groups for both Accounts Receivable (A/R) and Inventory.', metadata={'_additional': {'distance': 0.5666206}, 'category': 'Assets', 'slug': 'Assets_AssetInputs', 'title': 'Asset Inputs'}),
 Document(page_content='# Inventory\n## Category: Inventory\nInventory is unused material and supplies used to create finished product as well as unsold finished product.\nInventory includes Work in Process which will include allocated manufacturing labor as well as the direct materials and supplies of unfinished Finished Product.\nUnsold completed Finished Product Inventory includes allocated direct manufacturing labor as well as the direct materials and supplies.\nCash paid to create inventory (see Inventory - Future Terms Set-up) or User capitalized expenses (see Info Browser - Individually Scheduled Inc

In [9]:
vectorstore.similarity_search(query, k=10)

[Document(page_content='Question: How many different types, categories, or groups can the User list for both A/R and Inventory?\nAnswer: The User can list up to four different types, categories, or groups for both Accounts Receivable (A/R) and Inventory.', metadata={'_additional': {'distance': 0.5666206}, 'category': 'Assets', 'slug': 'Assets_AssetInputs', 'title': 'Asset Inputs'}),
 Document(page_content='# Inventory\n## Category: Inventory\nInventory is unused material and supplies used to create finished product as well as unsold finished product.\nInventory includes Work in Process which will include allocated manufacturing labor as well as the direct materials and supplies of unfinished Finished Product.\nUnsold completed Finished Product Inventory includes allocated direct manufacturing labor as well as the direct materials and supplies.\nCash paid to create inventory (see Inventory - Future Terms Set-up) or User capitalized expenses (see Info Browser - Individually Scheduled Inc

In [2]:
from langchain.llms import Cohere

llm = Cohere(model="d2cd24fd-67fd-4610-ab84-a37f9e635313-ft")

In [3]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

prompt_template="""You are an assistant specialized in ThruThink budgeting analysis and projection web application usage.
You are also knowledgeable in a wide range of budgeting and accounting topics, including EBITDA, cash flow balance, inventory management, and more.
While you strive to provide accurate information and assistance, please keep in mind that you are not a licensed investment advisor, financial advisor, or tax advisor.
Therefore, you cannot provide personalized investment advice, financial planning, or tax guidance.
You are here to offer general information, answer your questions, and assist with ThruThink-related inquiries to the best of your knowledge.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}"""

messages = [
    SystemMessagePromptTemplate.from_template(prompt_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)

In [4]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

chain = (
    {
        "summaries": itemgetter("summaries"),
        "question": itemgetter("question")
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [6]:
query = "How much inventory should I have?"
chain.invoke({"question": query, "summaries": ""})

" It depends on the product and the company. Generally, a small percentage of inventory is needed for financial reporting purposes, but the specific percentage should be determined by the company's inventory management needs and the level of inventory desired. It is important to consider the specific inventory needs of the company and the level of inventory desired when determining the appropriate inventory percentage."

In [7]:
import cohere
co = cohere.Client(cohere_api_key)

In [9]:
system_prompt = """You are an assistant specialized in ThruThink budgeting analysis and projection web application usage.
You are also knowledgeable in a wide range of budgeting and accounting topics, including EBITDA, cash flow balance, inventory management, and more.
While you strive to provide accurate information and assistance, please keep in mind that you are not a licensed investment advisor, financial advisor, or tax advisor.
Therefore, you cannot provide personalized investment advice, financial planning, or tax guidance.
You are here to offer general information, answer your questions, and assist with ThruThink-related inquiries to the best of your knowledge.
If you don't know the answer, just say that "I don't know", don't try to make up an answer."""

response = co.chat(
    message=query,
    prompt_truncation='auto',
    connectors=[{"id": "web-search"}],
    preamble_override=system_prompt
)

In [10]:
response

cohere.Chat {
	id: 14039cb7-ef0b-43fa-896d-bc3bad5b8929
	response_id: 14039cb7-ef0b-43fa-896d-bc3bad5b8929
	generation_id: aca1c93c-0baf-4708-a22f-9f8565cc902f
	message: How much inventory should I have?
	text: Businesses should always hold enough stock to meet unexpected spikes in consumer demand but not too much to cause overstock issues. The amount of inventory a business should hold depends on various factors such as customer demand, lead time to process orders, holding costs, and order costs. 

For example, if a business has a lead time of four weeks to process customer orders and their customers place orders for 100 pieces of a product twice a week, they should have enough inventory to cover 200 pieces to account for the lead time and expected orders. 

Additionally, businesses may choose to hold more inventory during peak seasons or periods, such as summer or holidays like Christmas, to account for increased customer demand. 

It's important to note that the appropriate inventor

In [6]:
import json

response = (
    client.query
    .get("Help", ["text", "title", "category"])
    .with_near_text({
        "concepts": ["What is ThruThink?"]
    })
    .with_limit(5) # limit the output to only 2
    .with_additional(["distance", "id"]).do()
)

print(json.dumps(response, indent=2))

{
  "data": {
    "Get": {
      "Help": [
        {
          "_additional": {
            "distance": 0.5238858,
            "id": "04399f50-e09d-4d85-b45c-39887cbcbe3d"
          },
          "category": "ThruThink",
          "text": "# Customer Assistance\n## Category: ThruThink\n\nEmail: Support@ThruThink.com\nCustomer Service Hours: Pacific Time\nMailing Address: 6061 N. Fresno Street, Suite 106, Fresno, California, USA\nEntity name: Business BrainScience, LLC\n",
          "title": "Customer Assistance"
        },
        {
          "_additional": {
            "distance": 0.5318928,
            "id": "04cae26b-8413-4bcc-ae21-3e668dee6b4e"
          },
          "category": "Summary",
          "text": "Question: What does the ThruThink analysis illustrate?\nAnswer: The ThruThink analysis illustrates the projected performance of a business in conjunction with its debt and equity structure.",
          "title": "Historical Worksheet - Before Beginning"
        },
        {
    